In [1]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils
from datetime import datetime
import IMMfunction
import JCfunc
import MMJfunction
from Map_Environment import ME

In [2]:
# gdf_utm = pd.read_pickle('gdf_utm.pkl')
# edges_utm = pd.read_pickle('edges_utm.pkl')
# nodes_utm = pd.read_pickle('nodes_utm.pkl')
# # curr_edge = pd.read_pickle('current_edge.pkl')

# # for debugging purposes start at iteration point 13
# point_index = 13
# # this is for debugging purposes, true 
# prev_loc = gdf_utm.iloc[point_index-1].to_frame().T
# last_matched = point_matching(prev_loc, curr_edge.iloc[0])


# # current location 
# curr_loc = gdf_utm.iloc[[point_index]]

# # find longitude and latitude for last matched data
# last_matched['lon_lat'] = last_matched.to_crs({'init': 'epsg:4326'})

In [3]:
# some functions
def get_bearing(point1, point2):
    # this code calculates the bearing of any given pair of longitude, latitude  
    geodesic = pyproj.Geod(ellps='WGS84')
    fwd_azimuth,back_azimuth,distance = geodesic.inv(point1[0], point1[1], point2[0], point2[1])
    return fwd_azimuth

def edge_bearing(edge):
    # this function calculates the bearing from the starting and ending node of each road segment
    bearing = get_bearing(edge[0], edge[len(edge) - 1])
    return bearing
    
def conv_angle(angle):
    # this function converts the angle from [-pi, pi] to [0, 2pi]
    if angle < 0 :
        angle = angle + 360
    return angle

def adjust_angle(angle):
    # this function converts the angle so that if the angle is greater than pi it replaced with 2pi-angle
    if angle > 180:
        angle = 360 - angle
    return angle

def conc(a):
    #function to convert list or integer in osmid into a unique string id 
    if type(a) is int:
        return str(a)
    ans = ",".join(map(str, a))
    return ans

def err_polygon(curr_loc, err_size):
    # function that output shapely polygon for point error bound
    x = curr_loc['geometry'].iloc[0].x
    y = curr_loc['geometry'].iloc[0].y
    
    err_coord = [[x - err_size, y + err_size], 
                 [x + err_size, y + err_size],
                 [x + err_size, y - err_size],
                 [x - err_size, y - err_size]]

    poly_coord = Polygon(err_coord)
    # #print(ply_coord)
    df = {'Attribute' : ['name1'], 'geometry':poly_coord}

    #projected to UTM 31 
    err_poly = gpd.GeoDataFrame(df, geometry = 'geometry', crs = "EPSG:32631")
    
    return err_poly

def check_connect(link):
    # this function check connectivity of candidate link with the current edge
    # needed to convert key to column and the previous end node saved in the dataframe
    # 1 if its connect 0 if elsewhere
    if (link['u'] == link['prev_end_node']):
        a = 1
    else : 
        a = 0
    return a

In [4]:
trajectory_data = pd.read_pickle('gdf_utm.pkl')
nodes_data = pd.read_pickle('nodes_utm.pkl')
edges_data = pd.read_pickle('edges_utm.pkl')

In [5]:
iter = 0

In [6]:
def IMM(trajectory_data, edges_data, iter):
    # initialization for IMM
    stop_iter = False
    err_size = 38
    
    # saving answer for debugging purposes 
    # edge_link saves all the candidate link name for each iteration 
    # final answer is stored in the edge_link variable 
    edge_link = []
    #curr_pos
    curr_pos_list = []
    # save candidate link name each iteration  
    candidate_link_res = []
    
    
    while stop_iter == False :
        # extract current location at given iteration 
        curr_loc = trajectory_data.iloc[iter].to_frame().T
        # save the iteration current position as a list
        curr_pos_list.append(curr_loc)
    
        # if the vehicle speed is less than 3m/s we skip it because the data at the time is less reliable
        if curr_loc['speed_mps'].iloc[0] < 3:
            # print(['the vehicle speed is less than 3m/s at iteration number', iter + 1])
            iter = iter + 1
        else:   
            # input should be location and error size 
            # create rectangular polygon 
            err_poly = err_polygon(curr_loc, err_size)
        
            # to plot error polygon for debugging
            # err_poly.plot()
            
            # Check for intersection and containment using geopandas
            intersects = gpd.sjoin(err_poly, edges_data, predicate='intersects')
            contains = gpd.sjoin(err_poly, edges_data, predicate='contains')
        
            if (len(intersects) + len(contains)) <= 0:
                # print(['no edeges intersects with error bound at iteration number', iter + 1])
                iter = iter + 1
            else:    
                stop_iter = True
                # perform IMP only when there is edge intersects with error bound
                # print(['edges found at iteration number', iter + 1])
        
                # extract index from edges that intersect with error polygon 
                int_index = intersects[['index_right0', 'index_right1', 'index_right2']]
                # extract index from edges that contained in the error polygon 
                cont_index = contains[['index_right0', 'index_right1', 'index_right2']]
        
                # merge index
                index = pd.concat([int_index, cont_index])
                # drop duplicate
                index = index.drop_duplicates()
        
                # initialize candidate edges 
                appended_edge = []
        
                # extract candidate eges  
                for i in range(len(index)):
                    edge_list = (index['index_right0'].iloc[i], index['index_right1'].iloc[i], 0 )
                    appended_edge.append(edge_list)
        
                candidate_link = edges_data.loc[appended_edge]
        
                #save candidate link name 
                candidate_link_res.append(candidate_link['osmid'])
        
                # calculate perpendicular distance 
                # initialize list that hold perpendicular distance between points and edges
                p_dist = []
        
                # calculate perpendicular distance between current point and candidate edges
                for i in range(len(candidate_link)):
                    p_dist.append(candidate_link['geometry'].iloc[i].distance(curr_loc['geometry']).iloc[0])
        
                # attach perpendicular distance to candidate link 
                candidate_link["perp_dist"] = p_dist
        
                # calculate heading error
                # convert lat lon into tuple coordinate 
                candidate_link['lon_lat_pair'] = candidate_link.lon_lat.apply(lambda geom: list(geom.coords))
        
                # calculate bearing from start and end node for each candidate link (see notes below)
                bearing_raw = candidate_link['lon_lat_pair'].apply(edge_bearing)
        
                # convert bearing from -pi, pi to 0, 2pi range
                candidate_link['edge_heading'] = bearing_raw.apply(conv_angle)
        
                # heading difference = abs(gps heading - edge bearing)
                heading_diff = abs(candidate_link['edge_heading'] - trajectory_data['GPS Bearing'].iloc[iter])
                
                # convert heading difference so that all its values lie from 0 to pi because the contribution of angle x and 2pi-x should be equal.
                candidate_link['heading_error'] = heading_diff.apply(adjust_angle)
        
        
                # input for IMM
                PD = candidate_link['perp_dist'].to_list()
                HE = candidate_link['heading_error'].to_list()
        
                # find the index corresponding to the highest weight edge
                imm_res = IMMfunction.IMMfunc(PD, HE, ME(edges_data, curr_loc))
                
                edge_link.append(candidate_link['osmid'].iloc[imm_res])

                # loc = np.where(edges_data["str_id"] == conc(edge_link[0]))
                # a = edges_data.iloc[loc]

                a = candidate_link.iloc[[imm_res]]

                iter = iter + 1

                return iter, a

In [7]:
iter, curr_loc = IMM(pd.read_pickle('gdf_utm.pkl'),pd.read_pickle('edges_utm.pkl'), iter)

2
529.3530000000001
0.7075325
2.8267252741040165
-----------------------------------------------------------------------------


In [7]:
for i in range(len(trajectory_data)):
    x = trajectory_data.iloc[i].to_frame().T
    print(ME(edges_data, x))

1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [8]:
for i in range(len(trajectory_data)):
    x = trajectory_data.iloc[i].to_frame().T
    print(ME(edges_data, x))

12
2
0.354237
5.645937606743508
-----------------------------------------------------------------------------
1
12
2
0.354237
5.645937606743508
-----------------------------------------------------------------------------
1
12
2
0.354237
5.645937606743508
-----------------------------------------------------------------------------
1
10
2
0.256352
7.801772562726251
-----------------------------------------------------------------------------
0
8
2
0.194379
10.289177328826673
-----------------------------------------------------------------------------
0
8
2
0.194379
10.289177328826673
-----------------------------------------------------------------------------
0
8
2
0.194379
10.289177328826673
-----------------------------------------------------------------------------
0
8
2
0.194379
10.289177328826673
-----------------------------------------------------------------------------
0
8
2
0.194379
10.289177328826673
------------------------------------------------------------------------

In [8]:
for i in range(len(trajectory_data)):
    x = trajectory_data.iloc[i].to_frame().T
    print(ME(edges_data, x))

2
diff 293.0729999999999
cont 354.237
int 647.31
total 0.5007735
3.993821558049697
-----------------------------------------------------------------------------
1
2
diff 293.0729999999999
cont 354.237
int 647.31
total 0.5007735
3.993821558049697
-----------------------------------------------------------------------------
1
2
diff 293.0729999999999
cont 354.237
int 647.31
total 0.5007735
3.993821558049697
-----------------------------------------------------------------------------
1
2
diff 390.9579999999999
cont 256.35200000000003
int 647.31
total 0.45183100000000004
4.426433777230867
-----------------------------------------------------------------------------
1
2
diff 248.47700000000006
cont 194.379
int 442.85600000000005
total 0.3186175
6.277119116181629
-----------------------------------------------------------------------------
1
2
diff 248.47700000000006
cont 194.379
int 442.85600000000005
total 0.3186175
6.277119116181629
-------------------------------------------------------

In [19]:
iter

12

In [20]:
curr_loc

,,,osmid,lanes,ref,name,highway,maxspeed,oneway,reversed,length,geometry,width,access,lon_lat,str_id,perp_dist,lon_lat_pair,edge_heading,heading_error
u,v,key,,,,,,,,,,,,,,,,,,
135535395,53200191,0,"[673214248, 234046511]",3,L 793,Wolbecker Straße,secondary,50,False,False,54.889,"LINESTRING (819640.807 5766244.782, 819631.704...",NaN,NaN,"LINESTRING (7.65261 51.95483, 7.65247 51.95483...","673214248,234046511",31.293527,"[(7.6526059, 51.9548309), (7.6524737, 51.95483...",273.998415,30.601591


In [9]:
def JC(trajectory_data, nodes_data, edges_data, current_position, previous_edge, iter):
        # extract node id
    edge_node1 = nodes_data.loc[previous_edge.index[0][0]]
    edge_node2 = nodes_data.loc[previous_edge.index[0][1]]
    
    # select the end node 
    if (current_position['GPS Bearing'].iloc[0] > 0 and current_position['GPS Bearing'].iloc[0] < 180):
        # if object movint to the right of map select the one with larger longitude
        if edge_node1.x > edge_node2.x:
            end_node = edge_node1
            start_node = edge_node2
        else:
            end_node = edge_node2
            start_node = edge_node1
    else:
        # if object is moving to the left select node with smaller longitude
        if edge_node1.x < edge_node2.x:
            end_node = edge_node1
            start_node = edge_node2
        else:
            end_node = edge_node2
            start_node = edge_node1
    
    # angle distance between the heading of the current point and the heading of the previous point
    # it should lie between 0 to pi
    delta_h = adjust_angle(abs(trajectory_data['GPS Bearing'].iloc[iter] - trajectory_data['GPS Bearing'].iloc[iter - 1]))
    
    # distance traveled from the last position fix to the end nodes
    d_1 = end_node['geometry'].distance(current_position['geometry']).iloc[0]
    
    # distance traveled since the last position fix 
    t = trajectory_data['time'].iloc[iter] - trajectory_data['time'].iloc[iter - 1]
    d_2 = (trajectory_data['speed_mps'].iloc[iter])* t.seconds
    
    delta_d = d_1 - d_2
    
    speed = current_position['speed_mps'].iloc[0]
    
    return JCfunc.MML(delta_d, delta_h, speed)

In [10]:
JC(pd.read_pickle('gdf_utm.pkl'), pd.read_pickle('nodes_utm.pkl'), pd.read_pickle('edges_utm.pkl'), pd.read_pickle('gdf_utm.pkl').iloc[iter].to_frame().T, curr_loc,iter)

False

In [11]:
iter

12

In [12]:
def MMJ(trajectory_data, nodes_data, edges_data, current_position, previous_edge, iter):
    err_size = 38
    # extract node id
    edge_node1 = nodes_data.loc[previous_edge.index[0][0]]
    edge_node2 = nodes_data.loc[previous_edge.index[0][1]]
    
    edge_link = []
    
    # select the end node 
    if (current_position['GPS Bearing'].iloc[0] > 0 and current_position['GPS Bearing'].iloc[0] < 180):
        # if object movint to the right of map select the one with larger longitude
        if edge_node1.x > edge_node2.x:
            end_node = edge_node1
            start_node = edge_node2
        else:
            end_node = edge_node2
            start_node = edge_node1
    else:
        # if object is moving to the left select node with smaller longitude
        if edge_node1.x < edge_node2.x:
            end_node = edge_node1
            start_node = edge_node2
        else:
            end_node = edge_node2
            start_node = edge_node1
    
    # get edges inside the error region 
    err_poly = err_polygon(current_position, err_size)
    
    intersects = gpd.sjoin(err_poly, edges_data, predicate='intersects')
    contains = gpd.sjoin(err_poly, edges_data, predicate='contains')
    print(len(intersects))
    
    if (len(intersects) + len(contains)) >0:
        # extract index from edges that intersect with error polygon 
        int_index = intersects[['index_right0', 'index_right1', 'index_right2']]
        # extract index from edges that contained in the error polygon 
        cont_index = contains[['index_right0', 'index_right1', 'index_right2']]
    
        # merge index
        index = pd.concat([int_index, cont_index])
        # drop duplicate
        index = index.drop_duplicates()
    
        # initialize candidate edges 
        appended_edge = []
    
        # extract candidate eges  
        for i in range(len(index)):
            edge_list = (index['index_right0'].iloc[i], index['index_right1'].iloc[i], 0 )
            appended_edge.append(edge_list)
    
        candidate_link = edges_data.loc[appended_edge]
        # store previous end link info for connectivity checking 
        candidate_link['prev_end_node'] = np.repeat(end_node.name, len(candidate_link))
        # put u and v into column for connectivity checking
        candidate_link_uv = candidate_link.reset_index()
        
        # calculate perpendicular distance 
        # initialize list that hold perpendicular distance between points and edges
        p_dist = []
        # initialize list that hold connectivity 
        conn = []
        # calculate perpendicular distance between current point and connectivity
        for i in range(len(candidate_link)):
            p_dist.append(candidate_link['geometry'].iloc[i].distance(current_position['geometry']).iloc[0])
            conn.append(check_connect(candidate_link_uv.iloc[i]))
        
        # attach perpendicular distance to candidate link 
        candidate_link['perp_dist'] = p_dist
        
        # attach connectivity 
        candidate_link['connectivity'] = conn
    
        # print(candidate_link)
    
        # calculate heading error
        # convert lat lon into tupple coordinate 
        candidate_link['lon_lat_pair'] = candidate_link.lon_lat.apply(lambda geom: list(geom.coords))
    
        # calculate bearing frome start and end node for each candidate link (see notes below)
        bearing_raw = candidate_link['lon_lat_pair'].apply(edge_bearing)
    
        # convert bearing from -pi, pi to 0, 2pi range
        candidate_link['edge_heading'] = bearing_raw.apply(conv_angle)
    
        # heading difference = abs(gps heading - edge bearing)
        heading_diff = abs(candidate_link['edge_heading'] - current_position['GPS Bearing'].iloc[0])
                
        # convert heading difference so that all its values lie from 0 to pi because the contribution of angle x and 2pi-x should be equal.
        candidate_link['heading_error'] = heading_diff.apply(adjust_angle)    
        
        # initialize input for MMJ
        PD = candidate_link['perp_dist'].to_list()
        HE = candidate_link['heading_error'].to_list()
        TR = candidate_link['connectivity'].to_list()
        
        mmj_res = MMJfunc.MMJ(PD, HE, TR, 0)
        edge_link.append(candidate_link['osmid'].iloc[mmj_res])

        # loc = np.where(edges_data["str_id"] == conc(edge_link[0]))
        # a = edges_data.iloc[loc]
        
        a = candidate_link.iloc[[mmj_res]]
        
        iter = iter + 1

        return iter, a
        

In [13]:
iter, curr_loc = MMJ(pd.read_pickle('gdf_utm.pkl'), pd.read_pickle('nodes_utm.pkl'), pd.read_pickle('edges_utm.pkl'), pd.read_pickle('gdf_utm.pkl').iloc[iter].to_frame().T, curr_loc, iter)

6


In [14]:
curr_loc

,,,osmid,lanes,ref,name,highway,maxspeed,oneway,reversed,length,geometry,width,access,lon_lat,str_id,prev_end_node,perp_dist,connectivity,lon_lat_pair,edge_heading,heading_error
u,v,key,,,,,,,,,,,,,,,,,,,,
53200191,6152925224,0,470244584,3,L 793,Wolbecker Straße,secondary,50,False,False,36.973,"LINESTRING (819585.705 5766245.100, 819574.575...",NaN,NaN,"LINESTRING (7.65181 51.95487, 7.65165 51.95487...",470244584,53200191,28.933227,1,"[(7.651807, 51.9548654), (7.6516457, 51.954873...",274.577083,22.922917
